In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from geospatial_4pointinpolygon_version import *

In [2]:
# Function: Extract coordinates value from pointgroup
# Input: a list of point, each point is of the point class from geospatial.py.
# Output: a numpy array list of coordinates.
def coordinate_list(pointgroup):
    n = len(pointgroup)
    valuelist = []
    for i in range(0, n):
        valuelist.append([pointgroup[i].x, pointgroup[i].y])
    return valuelist

# Function: calculate distance between each pair of points from two point group
# Input: a list of points, each point is of the point class from geospatial.py.
# Output: a matrix of distance value between each pair of points from the two point group.
def distance_matrix_from_two_pointgroup(pointgroup1, pointgroup2):
    n = len(pointgroup1)
    m = len(pointgroup2)
    distance_matrix = np.zeros((n, m))
    for i in range(0, n):
        for j in range(0, m):
            distance_matrix[i, j] = pointgroup1[i].distEuclidean(pointgroup2[j])
    return distance_matrix

# Function: Calculate the dtw value(dynamic time warping distance)
# Input: two list of points, each point is of the point class from geospatial.py.
# Output 1: d[-1,-1], which is the dtw distance value, a float.
# Output 2: d, the cumulative distance matrix
def dtw(pointgroup1, pointgroup2):
    d_matrix = distance_matrix_from_two_pointgroup(pointgroup1, pointgroup2)
    d = np.zeros(d_matrix.shape)
    d[0, 0] = d_matrix[0, 0]
    n, m = d_matrix.shape
    for i in range(1, n):
        d[i, 0] = d[i-1, 0] + d_matrix[i, 0]
    for j in range(1, m):
        d[0, j] = d[0, j-1] + d_matrix[0, j]
    for i in range(1, n):
        for j in range(1, m):
            d[i, j] = d_matrix[i, j] + min((d[i-1, j], d[i, j-1], d[i-1, j-1]))
    print(f"dtw value: {d[-1, -1]}")
    print(f"dtw distance matrix: {d}")
    return d[-1, -1], d

# Function: Calculate the optimal dtw path of a given dtw cumulative distance matrix
# Input: a matrix, specifically a dtw cumulative distance matrix (result d of dtw function).
# Output: a ndarray, the shortest/optimal path of the distance matrix from the first cell to the last one.
def dtw_path(d):
    path = []
    i, j = d.shape
    i = i - 1
    j = j - 1
    path.append((i, j))
    while i > 0 or j > 0:
        if i == 0:
            j = j - 1
        elif j == 0:
            i = i - 1
        else:
            temp_step = min([d[i-1, j], d[i, j-1], d[i-1, j-1]])
            if d[i-1, j] == temp_step:
                i = i - 1
            elif d[i, j-1] == temp_step:
                j = j - 1
            else:
                i = i - 1
                j = j - 1
        path.append((i, j))
    path = np.array(path)
    # reverse the order of path, such that it starts with [0, 0]
    return path[::-1]